In [55]:
import pandas as pd
from configparser import ConfigParser
import psycopg2
import pandas.io.sql as sqlio
from sqlalchemy import create_engine
import numpy as np

In [56]:
# Read in csv files
dce_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_dce.csv")
#fcev_df=pd.read_csv("..Data/NREL 2019 Survey Data/data/survey_res_fcev.csv")
main_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_main.csv")
person_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_person.csv")
pev_df=pd.read_csv("../Data/NREL 2019 Survey Data/data/survey_res_pev.csv")
vehicle_df=pd.read_csv("../Data//NREL 2019 Survey Data/data/survey_res_vehicle.csv")

## OVERVIEW OF DATA TABLES

In [57]:
# Discrete Choice Experiment table
print(dce_df.shape)
dce_df.head()

(33984, 89)


,sampno,sequence,alt_order,flag_dce_year,alt_1_vehtype,alt_2_vehtype,alt_3_vehtype,alt_4_vehtype,alt_1_fueltype,alt_2_fueltype,...,alt_4_purchincent,alt_1_anncost,alt_2_anncost,alt_3_anncost,alt_4_anncost,alt_1_accel,alt_2_accel,alt_3_accel,alt_4_accel,choice
0,190329,1,"3,1,2,4",2025,8,11,12,3,1,3,...,1,1660,747,996,646,8.5,5.2,5.9,8.9,1
1,190329,2,"3,1,2,4",2025,8,6,8,11,3,1,...,1,1494,799,2075,830,8.1,8.2,8.5,5.5,1
2,190329,3,"3,1,2,4",2025,8,8,9,2,2,8,...,1,2490,1660,1081,1153,7.5,6.8,5.9,8.3,2
3,190329,4,"3,1,2,4",2025,8,8,10,11,8,7,...,1,1660,1037,403,830,10.2,6.5,7.9,5.5,1
4,190329,5,"3,1,2,4",2025,4,4,7,11,2,7,...,1,1844,1153,2213,692,5.1,6.6,6.5,5.5,1


In [58]:
print(dce_df.dtypes)
dce_df.nunique()

sampno             int64
sequence           int64
alt_order         object
flag_dce_year      int64
alt_1_vehtype      int64
                  ...   
alt_1_accel      float64
alt_2_accel      float64
alt_3_accel      float64
alt_4_accel      float64
choice             int64
Length: 89, dtype: object


sampno           4248
sequence            8
alt_order          24
flag_dce_year       2
alt_1_vehtype      13
                 ... 
alt_1_accel        98
alt_2_accel        98
alt_3_accel        98
alt_4_accel        98
choice              4
Length: 89, dtype: int64

In [59]:
dce_df.count()

sampno           33984
sequence         33984
alt_order        33984
flag_dce_year    33984
alt_1_vehtype    33984
                 ...  
alt_1_accel      33984
alt_2_accel      33984
alt_3_accel      33984
alt_4_accel      33984
choice           33984
Length: 89, dtype: int64

In [60]:
#print(fcev_df.shape)
#fcev_df.head()

In [61]:
#print(fcev_df.dtypes)
#print(fcev_df.nunique())

In [62]:
# Main demographics table
print(main_df.shape)
main_df.head()

(4248, 304)


,sampno,source,language,age_grp,california,county,region,future_decision_role,num_hh_vehicles,household_members_1,...,autonomous_att_8,autonomous_hhveh,autonomous_rideshare,autonomous_pooled,autonomous_pref,housing,housing_1_8_x,solar,solar_future,income
0,192850,4,1,3,1,37,4,1,2,0,...,2,2,3,3,1,1,,2,1,11
1,193449,4,1,3,1,34,6,3,3,0,...,3,2,2,2,3,1,,1,,8
2,193440,4,1,3,1,50,1,1,2,0,...,4,2,3,2,1,1,,2,2,11
3,193133,4,1,4,1,33,2,1,1,0,...,2,2,1,2,3,1,,2,2,7
4,193400,4,1,3,1,1,5,3,3,0,...,4,2,2,2,2,1,,2,2,7


In [63]:
# Create list of columns to drop for ML model
col_drop=[
    'vehicletype_consideration_1',
    'vehicletype_consideration_2',
    'vehicletype_consideration_3',
    'vehicletype_consideration_4',
    'vehicletype_consideration_5',
    'vehicletype_consideration_6',
    'vehicletype_consideration_7',
    'vehicletype_consideration_8',
    'vehicletype_consideration_9',
    'vehicletype_consideration_10',
    'vehicletype_consideration_11',
    'vehicletype_consideration_12',
    'vehicletype_consideration_13',
    'fueltype_consideration_1_1',
    'fueltype_consideration_2_1',
    'fueltype_consideration_3_1',
    'fueltype_consideration_4_1',
    'fueltype_consideration_5_1',
    'fueltype_consideration_6_1',
    'fueltype_consideration_7_1',
    'fueltype_consideration_8_1',
    'fueltype_consideration_9_1',
    'fueltype_consideration_10_1',
    'fueltype_consideration_11_1',
    'fueltype_consideration_12_1',
    'fueltype_consideration_13_1',
    'fueltype_consideration_1_2',
    'fueltype_consideration_2_2',
    'fueltype_consideration_3_2',
    'fueltype_consideration_4_2',
    'fueltype_consideration_5_2',
    'fueltype_consideration_6_2',
    'fueltype_consideration_7_2',
    'fueltype_consideration_8_2',
    'fueltype_consideration_9_2',
    'fueltype_consideration_10_2',
    'fueltype_consideration_11_2',
    'fueltype_consideration_12_2',
    'fueltype_consideration_13_2',
    'fueltype_consideration_1_3',
    'fueltype_consideration_2_3',
    'fueltype_consideration_3_3',
    'fueltype_consideration_4_3',
    'fueltype_consideration_5_3',
    'fueltype_consideration_6_3',
    'fueltype_consideration_7_3',
    'fueltype_consideration_8_3',
    'fueltype_consideration_9_3',
    'fueltype_consideration_10_3',
    'fueltype_consideration_11_3',
    'fueltype_consideration_12_3',
    'fueltype_consideration_13_3',
    'fueltype_consideration_1_4',
    'fueltype_consideration_2_4',
    'fueltype_consideration_3_4',
    'fueltype_consideration_4_4',
    'fueltype_consideration_5_4',
    'fueltype_consideration_6_4',
    'fueltype_consideration_7_4',
    'fueltype_consideration_8_4',
    'fueltype_consideration_9_4',
    'fueltype_consideration_10_4',
    'fueltype_consideration_11_4',
    'fueltype_consideration_12_4',
    'fueltype_consideration_13_4',
    'fueltype_consideration_1_5',
    'fueltype_consideration_2_5',
    'fueltype_consideration_3_5',
    'fueltype_consideration_4_5',
    'fueltype_consideration_5_5',
    'fueltype_consideration_6_5',
    'fueltype_consideration_7_5',
    'fueltype_consideration_8_5',
    'fueltype_consideration_9_5',
    'fueltype_consideration_10_5',
    'fueltype_consideration_11_5',
    'fueltype_consideration_12_5',
    'fueltype_consideration_13_5',
    'fueltype_consideration_1_6',
    'fueltype_consideration_2_6',
    'fueltype_consideration_3_6',
    'fueltype_consideration_4_6',
    'fueltype_consideration_5_6',
    'fueltype_consideration_6_6',
    'fueltype_consideration_7_6',
    'fueltype_consideration_8_6',
    'fueltype_consideration_9_6',
    'fueltype_consideration_10_6',
    'fueltype_consideration_11_6',
    'fueltype_consideration_12_6',
    'fueltype_consideration_13_6',
    'fueltype_consideration_1_7',
    'fueltype_consideration_2_7',
    'fueltype_consideration_3_7',
    'fueltype_consideration_4_7',
    'fueltype_consideration_5_7',
    'fueltype_consideration_6_7',
    'fueltype_consideration_7_7',
    'fueltype_consideration_8_7',
    'fueltype_consideration_9_7',
    'fueltype_consideration_10_7',
    'fueltype_consideration_11_7',
    'fueltype_consideration_12_7',
    'fueltype_consideration_13_7',
    'fueltype_consideration_1_8',
    'fueltype_consideration_2_8',
    'fueltype_consideration_3_8',
    'fueltype_consideration_4_8',
    'fueltype_consideration_5_8',
    'fueltype_consideration_6_8',
    'fueltype_consideration_7_8',
    'fueltype_consideration_8_8',
    'fueltype_consideration_9_8',
    'fueltype_consideration_10_8',
    'fueltype_consideration_11_8',
    'fueltype_consideration_12_8',
    'fueltype_consideration_13_8',
    'consideration_1',
    'consideration_2',
    'consideration_3',
    'consideration_4',
    'consideration_5',
    'consideration_6',
    'consideration_7',
    'consideration_8',
    'consideration_9',
    'consideration_10',
    'consideration_11',
    'consideration_12',
    'consideration_13',
    'consideration_14',
    'consideration_15',
    'consideration_16',
    'consideration_17',
    'consideration_18',
    'consideration_19',
    'consideration_20',
    'consideration_21',
    'consideration_22',
    'consideration_23',
    'consideration_24',
    'consideration_25',
    'consideration_26',
    'consideration_27',
    'consideration_28',
    'consideration_29',
    'consideration_30',
    'consideration_31',
    'consideration_32',
    'consideration_33',
    'consideration_34',
    'consideration_35',
    'consideration_36',
    'consideration_37',
    'consideration_38',
    'consideration_39',
    'consideration_40',
    'consideration_41',
    'consideration_42',
    'consideration_43',
    'consideration_44',
    'consideration_45',
    'consideration_46',
    'consideration_47',
    'consideration_48',
    'consideration_49',
    'consideration_50',
    'consideration_51',
    'consideration_52',
    'consideration_53',
    'consideration_54',
    'consideration_55',
    'consideration_56',
    'consideration_57',
    'consideration_58',
    'consideration_59',
    'consideration_60',
    'consideration_61',
    'consideration_62',
    'consideration_63',
    'consideration_64',
    'consideration_65',
    'consideration_66',
    'consideration_67',
    'consideration_68',
    'consideration_69',
    'consideration_70',
    'consideration_71',
    'consideration_72',
    'consideration_73',
    'consideration_74',
    'consideration_75',
    'consideration_76',
    'consideration_77',
    'consideration_78',
    'consideration_79',
    'consideration_80',
    'consideration_81',
    'consideration_82',
    'consideration_83',
    'consideration_84',
    'consideration_85',
    'consideration_86',
    'consideration_87',
    'consideration_88',
    'consideration_89',
    'consideration_90',
    'consideration_91',
    'consideration_92',
    'consideration_93',
    'consideration_94',
    'consideration_95',
    'consideration_96',
    'consideration_97',
    'consideration_98',
    'consideration_99',
    'consideration_100',
    'consideration_101',
    'consideration_102',
    'consideration_103',
    'consideration_104']
main_df.drop(columns=col_drop, inplace=True, axis=1, errors='ignore')
main_df.head()

,sampno,source,language,age_grp,california,county,region,future_decision_role,num_hh_vehicles,household_members_1,...,autonomous_att_8,autonomous_hhveh,autonomous_rideshare,autonomous_pooled,autonomous_pref,housing,housing_1_8_x,solar,solar_future,income
0,192850,4,1,3,1,37,4,1,2,0,...,2,2,3,3,1,1,,2,1,11
1,193449,4,1,3,1,34,6,3,3,0,...,3,2,2,2,3,1,,1,,8
2,193440,4,1,3,1,50,1,1,2,0,...,4,2,3,2,1,1,,2,2,11
3,193133,4,1,4,1,33,2,1,1,0,...,2,2,1,2,3,1,,2,2,7
4,193400,4,1,3,1,1,5,3,3,0,...,4,2,2,2,2,1,,2,2,7


In [64]:
main_df.fillna(0)

,sampno,source,language,age_grp,california,county,region,future_decision_role,num_hh_vehicles,household_members_1,...,autonomous_att_8,autonomous_hhveh,autonomous_rideshare,autonomous_pooled,autonomous_pref,housing,housing_1_8_x,solar,solar_future,income
0,192850,4,1,3,1,37,4,1,2,0,...,2,2,3,3,1,1,,2,1,11
1,193449,4,1,3,1,34,6,3,3,0,...,3,2,2,2,3,1,,1,,8
2,193440,4,1,3,1,50,1,1,2,0,...,4,2,3,2,1,1,,2,2,11
3,193133,4,1,4,1,33,2,1,1,0,...,2,2,1,2,3,1,,2,2,7
4,193400,4,1,3,1,1,5,3,3,0,...,4,2,2,2,2,1,,2,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243,190256,2,1,3,1,19,2,2,3,1,...,3,1,3,3,2,1,,2,2,7
4244,191249,5,1,3,1,30,2,3,3,0,...,3,1,3,2,3,1,,1,,9
4245,190327,2,1,3,1,36,2,2,3,0,...,3,2,2,3,3,1,,1,,9
4246,191658,5,1,4,1,19,2,2,3,1,...,3,2,3,3,3,1,,2,2,8


In [65]:
main_df.select_dtypes(include='object').columns.tolist()
#main_objcol.to_dict(orient='index')
#main_objcol_df=pd.DataFrame(main_objcol, index=None)
#replace_main=main_objcol_df.to_dict()

['hybrid_experience',
 'past_hybrid',
 'phev_experience',
 'past_phev',
 'bev_experience',
 'past_bev',
 'fcv_experience',
 'past_fcv',
 'charge_spots',
 'charge_work',
 'home_parking_1',
 'home_parking_2',
 'home_parking_3',
 'home_parking_4',
 'home_parking_5',
 'home_parking_6',
 'home_parking_7',
 'home_parking_8',
 'home_parking_9',
 'home_parking_8_x',
 'home_electricity_access',
 'hsa_distance_1_1_x',
 'hsa_distance_1_2_x',
 'future_purchase',
 'next_purchase',
 'purchase_timing',
 'mode_freq_1',
 'mode_freq_2',
 'mode_freq_3',
 'mode_freq_4',
 'mode_freq_5',
 'mode_freq_6',
 'mode_freq_7',
 'mode_freq_8',
 'mode_freq_9',
 'mode_freq_10',
 'mode_freq_11',
 'mode_freq_12',
 'autonomous_rideshare',
 'housing_1_8_x',
 'solar_future']

In [66]:
main_df.nunique()

sampno           4248
source              6
language            2
age_grp             3
california          1
                 ... 
housing             8
housing_1_8_x      21
solar               2
solar_future        3
income             11
Length: 83, dtype: int64

In [67]:
# Driver Demographics table
print(person_df.shape)
person_df.head()

(8365, 29)


,sampno,perid,perno,relationship,gender,employment,student,education,license,vehno,...,school_distance,ethnicity,race_1,race_2,race_3,race_4,race_5,race_6,race_7,race_6_x
0,190001,190001.p01,1,8,1,4,4,6,1,190001.v01,...,,2,0,0,0,0,1,0,0,
1,190003,190003.p01,1,8,2,4,4,8,1,190003.v01,...,,2,0,0,0,0,1,0,0,
2,190002,190002.p01,1,8,2,4,4,1,2,,...,,2,0,0,0,0,1,0,0,
3,190005,190005.p01,1,8,1,4,4,6,1,190005.v02,...,,2,0,0,0,0,1,0,0,
4,190002,190002.p02,2,1,1,1,4,2,1,,...,,2,0,0,0,0,1,0,0,


In [68]:
person_df.select_dtypes(include='object').columns.tolist()

['perid',
 'vehno',
 'vehicle_string',
 'drive_freq',
 'job_type',
 'work_mode',
 'work_distance',
 'work_days',
 'school_mode',
 'school_distance',
 'race_6_x']

In [69]:
person_df.nunique()

sampno             4248
perid              8365
perno                 9
relationship          8
gender                4
employment            5
student               4
education             8
license               2
vehno              6679
vehicle_string        1
drive_freq            5
transit_freq         34
tnc_freq             21
job_type              5
work_mode            17
work_distance       123
work_days             9
school_mode          13
school_distance      59
ethnicity             3
race_1                2
race_2                2
race_3                2
race_4                2
race_5                2
race_6                2
race_7                2
race_6_x            127
dtype: int64

In [70]:
person_df.fillna(0)

,sampno,perid,perno,relationship,gender,employment,student,education,license,vehno,...,school_distance,ethnicity,race_1,race_2,race_3,race_4,race_5,race_6,race_7,race_6_x
0,190001,190001.p01,1,8,1,4,4,6,1,190001.v01,...,,2,0,0,0,0,1,0,0,
1,190003,190003.p01,1,8,2,4,4,8,1,190003.v01,...,,2,0,0,0,0,1,0,0,
2,190002,190002.p01,1,8,2,4,4,1,2,,...,,2,0,0,0,0,1,0,0,
3,190005,190005.p01,1,8,1,4,4,6,1,190005.v02,...,,2,0,0,0,0,1,0,0,
4,190002,190002.p02,2,1,1,1,4,2,1,,...,,2,0,0,0,0,1,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8360,190802,190802.p02,2,1,2,4,4,8,1,190802.v03,...,,2,0,0,0,0,1,0,0,
8361,190845,190845.p01,1,8,2,2,4,5,1,190845.v01,...,,2,0,0,0,0,1,0,0,
8362,190803,190803.p01,1,8,2,1,4,8,1,190803.v01,...,,2,0,0,0,0,1,0,0,
8363,190806,190806.p01,1,8,1,1,4,2,1,190806.v01,...,,1,0,0,0,0,1,0,0,


In [71]:
# EV Owner Vehicle & Purchase information
print(pev_df.shape)
pev_df.head()

(451, 113)


,sampno,vehno,year,make,model,fuel,veh_type,pev_miles_week,pev_miles_month,phev_percent,...,pev_incentives_2,pev_incentives_3,pev_incentives_4,pev_incentives_5,pev_incentives_6,pev_incentives_7,pev_experience,pev_recommend,pev_confidence,pev_confidence_4_x
0,190107,1,2017,FORD,REDACTED,3,3,250,,30,...,5,4,4,1,1,4,5,5,,
1,190111,3,2018,VOLKSWAGEN,NaN,5,1,175,,,...,5,5,5,1,1,4,5,4,,
2,190120,2,2016,NISSAN,NaN,5,2,160,,,...,4,4,4,2,1,2,7,5,,
3,190127,1,2013,TOYOTA,NaN,3,3,200,,85,...,7,7,7,2,7,5,7,4,,
4,190138,3,2017,CHEVROLET,NaN,5,2,200,,,...,5,5,5,1,1,1,7,5,2,


In [72]:
pev_df.select_dtypes(include='object').columns.tolist()

['make',
 'model',
 'pev_miles_week',
 'pev_miles_month',
 'phev_percent',
 'recharge_amount',
 'home_charge_type_1',
 'home_charge_type_2',
 'home_charge_type_3',
 'home_charge_type_4',
 'home_charge_type_5',
 'home_charge_type_6',
 'home_charge_type_7',
 'home_charge_type_8',
 'home_charge_type_8_x',
 'home_charge_type_no_1',
 'home_charge_type_no_2',
 'home_charge_type_no_3',
 'home_charge_type_no_4',
 'home_charge_type_no_5',
 'home_charge_type_no_6',
 'home_charge_type_no_7',
 'home_charge_type_no_8',
 'home_charge_type_no_8_x',
 'charging_tech_1_1',
 'charging_tech_2_1',
 'charging_tech_3_1',
 'charging_tech_4_1',
 'charging_tech_5_1',
 'charging_tech_6_1',
 'charging_tech_7_1',
 'charging_tech_8_1',
 'charging_tech_1_2',
 'charging_tech_2_2',
 'charging_tech_3_2',
 'charging_tech_4_2',
 'charging_tech_5_2',
 'charging_tech_6_2',
 'charging_tech_7_2',
 'charging_tech_8_2',
 'charging_tech_1_3',
 'charging_tech_2_3',
 'charging_tech_3_3',
 'charging_tech_4_3',
 'charging_tech_5_3'

In [73]:
pev_df.nunique()

sampno                451
vehno                   7
year                   10
make                   19
model                   1
                     ... 
pev_incentives_7        7
pev_experience          7
pev_recommend           5
pev_confidence          5
pev_confidence_4_x      3
Length: 113, dtype: int64

In [74]:
pev_df.groupby(pev_df['fuel']).count()

,sampno,vehno,year,make,model,veh_type,pev_miles_week,pev_miles_month,phev_percent,battery_miles,...,pev_incentives_2,pev_incentives_3,pev_incentives_4,pev_incentives_5,pev_incentives_6,pev_incentives_7,pev_experience,pev_recommend,pev_confidence,pev_confidence_4_x
fuel,,,,,,,,,,,,,,,,,,,,,
3,173,173,173,173,1,173,173,173,173,173,...,173,173,173,173,173,173,173,173,173,173
5,278,278,278,278,0,278,278,278,278,278,...,278,278,278,278,278,278,278,278,278,278


In [75]:
pev_df.fillna(0)

,sampno,vehno,year,make,model,fuel,veh_type,pev_miles_week,pev_miles_month,phev_percent,...,pev_incentives_2,pev_incentives_3,pev_incentives_4,pev_incentives_5,pev_incentives_6,pev_incentives_7,pev_experience,pev_recommend,pev_confidence,pev_confidence_4_x
0,190107,1,2017,FORD,REDACTED,3,3,250,,30,...,5,4,4,1,1,4,5,5,,
1,190111,3,2018,VOLKSWAGEN,0,5,1,175,,,...,5,5,5,1,1,4,5,4,,
2,190120,2,2016,NISSAN,0,5,2,160,,,...,4,4,4,2,1,2,7,5,,
3,190127,1,2013,TOYOTA,0,3,3,200,,85,...,7,7,7,2,7,5,7,4,,
4,190138,3,2017,CHEVROLET,0,5,2,200,,,...,5,5,5,1,1,1,7,5,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,194232,1,2016,FORD,0,3,3,200,,20,...,7,7,7,7,1,2,7,5,,
447,194237,1,2013,CHEVROLET,0,3,2,40,,10,...,4,4,4,1,1,1,7,5,,
448,194239,1,2018,TESLA,0,5,3,250,,,...,5,5,7,1,7,4,7,5,,
449,194242,2,2016,TESLA,0,5,3,70,,,...,2,2,2,2,2,2,5,3,,


In [76]:
# Vehicle information table
print(vehicle_df.shape)
vehicle_df.head()

(8049, 46)


,sampno,vehno,vehicle_num,num_hh_vehicles,year,make,make_other,make_clean,model,model_other,...,personal_miles,business_miles,company_primary_driver,company_primary_driver_id,other_drivers,behavior_1,behavior_2,behavior_3,behavior_4,behavior_5
0,190001,1,1,1,1998,TOYOTA,,TOYOTA,REDACTED,REDACTED,...,,,,,,,,,,
1,190003,1,1,2,2015,LEXUS,,LEXUS,NaN,NaN,...,,,,,,,,,,
2,190005,2,2,2,2002,CHRYSLER,,CHRYSLER,NaN,NaN,...,,,,,,,,,,
3,190006,1,1,2,2018,TOYOTA,,TOYOTA,NaN,NaN,...,,,,,,,,,,
4,190009,3,3,3,2002,HONDA,,HONDA,NaN,NaN,...,,,,,,,,,,


In [77]:
pev_df.select_dtypes(include='object').columns.tolist()

['make',
 'model',
 'pev_miles_week',
 'pev_miles_month',
 'phev_percent',
 'recharge_amount',
 'home_charge_type_1',
 'home_charge_type_2',
 'home_charge_type_3',
 'home_charge_type_4',
 'home_charge_type_5',
 'home_charge_type_6',
 'home_charge_type_7',
 'home_charge_type_8',
 'home_charge_type_8_x',
 'home_charge_type_no_1',
 'home_charge_type_no_2',
 'home_charge_type_no_3',
 'home_charge_type_no_4',
 'home_charge_type_no_5',
 'home_charge_type_no_6',
 'home_charge_type_no_7',
 'home_charge_type_no_8',
 'home_charge_type_no_8_x',
 'charging_tech_1_1',
 'charging_tech_2_1',
 'charging_tech_3_1',
 'charging_tech_4_1',
 'charging_tech_5_1',
 'charging_tech_6_1',
 'charging_tech_7_1',
 'charging_tech_8_1',
 'charging_tech_1_2',
 'charging_tech_2_2',
 'charging_tech_3_2',
 'charging_tech_4_2',
 'charging_tech_5_2',
 'charging_tech_6_2',
 'charging_tech_7_2',
 'charging_tech_8_2',
 'charging_tech_1_3',
 'charging_tech_2_3',
 'charging_tech_3_3',
 'charging_tech_4_3',
 'charging_tech_5_3'

In [78]:
vehicle_df.nunique()

sampno                       4136
vehno                           8
vehicle_num                     8
num_hh_vehicles                 8
year                           40
make                           53
make_other                     36
make_clean                     69
model                           1
model_other                     1
model_clean                     1
fuel                            8
fuel_clean                      8
veh_type                       13
veh_type_clean                 13
vehicle_status                  5
flag_hev                        2
flag_phev                       2
flag_fcev                       2
flag_bev                        2
obtain                          6
company_car                     3
current_acquired_year          40
current_acquired_month         13
annual_mileage                281
electric_percent               37
tnc_veh                         3
tnc_miles                      51
delivery                        3
delivery_miles

In [79]:
vehicle_df.fillna(0)

,sampno,vehno,vehicle_num,num_hh_vehicles,year,make,make_other,make_clean,model,model_other,...,personal_miles,business_miles,company_primary_driver,company_primary_driver_id,other_drivers,behavior_1,behavior_2,behavior_3,behavior_4,behavior_5
0,190001,1,1,1,1998,TOYOTA,,TOYOTA,REDACTED,REDACTED,...,,,,,,,,,,
1,190003,1,1,2,2015,LEXUS,,LEXUS,0,0,...,,,,,,,,,,
2,190005,2,2,2,2002,CHRYSLER,,CHRYSLER,0,0,...,,,,,,,,,,
3,190006,1,1,2,2018,TOYOTA,,TOYOTA,0,0,...,,,,,,,,,,
4,190009,3,3,3,2002,HONDA,,HONDA,0,0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8044,192322,2,2,2,2013,SUBARU,,SUBARU,0,0,...,,,,,,,,,,
8045,192327,2,2,2,2018,TESLA,,TESLA,0,0,...,,,,,,,,,,
8046,192334,4,4,4,2013,SCION,,SCION,0,0,...,,,,,,,,,,
8047,192330,2,2,2,2015,FORD,,FORD,0,0,...,,,,,,,,,,


In [80]:
vehicle_df.groupby(vehicle_df['fuel_clean']).count()

,sampno,vehno,vehicle_num,num_hh_vehicles,year,make,make_other,make_clean,model,model_other,...,personal_miles,business_miles,company_primary_driver,company_primary_driver_id,other_drivers,behavior_1,behavior_2,behavior_3,behavior_4,behavior_5
fuel_clean,,,,,,,,,,,,,,,,,,,,,
1,6338,6338,6338,6338,6338,6338,6338,6338,1,1,...,6338,6338,6338,6338,6338,6338,6338,6338,6338,6338
2,587,587,587,587,587,587,587,587,0,0,...,587,587,587,587,587,587,587,587,587,587
3,220,220,220,220,220,220,220,220,0,0,...,220,220,220,220,220,220,220,220,220,220
4,118,118,118,118,118,118,118,118,0,0,...,118,118,118,118,118,118,118,118,118,118
5,333,333,333,333,333,333,333,333,0,0,...,333,333,333,333,333,333,333,333,333,333
6,306,306,306,306,306,306,306,306,0,0,...,306,306,306,306,306,306,306,306,306,306
8,140,140,140,140,140,140,140,140,0,0,...,140,140,140,140,140,140,140,140,140,140
9,7,7,7,7,7,7,7,7,0,0,...,7,7,7,7,7,7,7,7,7,7


## CONNECT TO POSTGRES AND UPDATE TABLES

In [81]:
from config import password

param_dic={
    "host": "localhost",
    "database": "Energy_Analysis",
    "user" : "postgres",
    "password" : password
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [82]:
connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)
    
def to_alchemy(df, table):
    """""
    Push dataframe to postgres table
    """
    engine = create_engine(connect)
    df.to_sql(
        table, 
        con=engine, 
        index=False, 
        if_exists='replace'
    )
    print("to_sql update complete")

In [83]:
to_alchemy(main_df, 'tbl_main')

to_sql update complete


In [84]:
to_alchemy(dce_df, 'tbl_dce')

to_sql update complete


In [85]:
to_alchemy(person_df, 'tbl_person')

to_sql update complete


In [86]:
to_alchemy(pev_df, 'tbl_pev')

to_sql update complete


In [87]:
to_alchemy(vehicle_df, 'tbl_vehicle')

to_sql update complete
